In [18]:
import pandas as pd

data = pd.read_csv("../../data/cleaned_data.csv")
data.head()

,text,label
0,នាយិកា មជ្ឈមណ្ឌល សិទ្ធិ មនុស្ស កម្ពុជា អ្នកស្រ...,neutral
1,ការឃុំ កញ្ញា សេង ធារី កាន់តែ យូរ រដ្ឋាភិបាល ហ៊...,positive
2,ប្រភព បង្ហើប បន្ទប់ ខ្ទង់ ចំណាយ ជាង ១០ម៉ឺន ដុល...,neutral
3,1956 បាន បង្ហាញ ផូស្វ័រ បាន ផ្ទេរ ដើម បែក អារ ...,neutral
4,ដរាបណា មិន បាន តាំងចិត្ត ខិតខំ ប្រឹង រៀន ប្រឹង...,negative


In [19]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
# Features and labels
X = data['text']  # cleaned text
y = data['label']      # labels

# Split dataset: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # stratify keeps class distribution
)


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Vectorize text
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X_train)  # X_texts = list of Khmer sentences

# 2. Oversample
ros = RandomOverSampler(random_state=42)
# Convert sparse matrix to dense temporarily
X_dense = X_vec.toarray()
X_res, y_res = ros.fit_resample(X_dense, y_train)

# 3. Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42, stratify=y_res)

# 4. Define models
models = {
    "MultinomialNB": MultinomialNB(),
    "LogisticRegression": LogisticRegression(max_iter=500, class_weight='balanced'),
    "LinearSVC": LinearSVC(class_weight='balanced', max_iter=1000),
    "RandomForest": RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    "DecisionTree": DecisionTreeClassifier(class_weight='balanced', random_state=42),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=100, random_state=42)
}

# 5. Train & evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print(f"=== {name} ===")
    print("Accuracy:", acc)
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    print("Confusion Matrix:")
    print(cm)

    print("-" * 50)

=== MultinomialNB ===
Accuracy: 0.6497498213009293

Classification Report:
              precision    recall  f1-score   support

    negative       0.59      0.65      0.62       932
     neutral       0.73      0.69      0.71       933
    positive       0.63      0.60      0.62       933

    accuracy                           0.65      2798
   macro avg       0.65      0.65      0.65      2798
weighted avg       0.65      0.65      0.65      2798

Confusion Matrix:
[[609 125 198]
 [161 645 127]
 [254 115 564]]
--------------------------------------------------
=== LogisticRegression ===
Accuracy: 0.6876340243030736

Classification Report:
              precision    recall  f1-score   support

    negative       0.65      0.68      0.66       932
     neutral       0.73      0.81      0.77       933
    positive       0.68      0.57      0.62       933

    accuracy                           0.69      2798
   macro avg       0.69      0.69      0.68      2798
weighted avg       0.69

In [21]:
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

results = []

for name, model in models.items():
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    macro_f1 = f1_score(y_test, y_pred, average="macro")

    results.append({
        "model": name,
        "accuracy": acc,
        "macro_f1": macro_f1
    })

df_results = pd.DataFrame(results)
print(df_results.sort_values("macro_f1", ascending=False))


                model  accuracy  macro_f1
3        RandomForest  0.875268  0.873013
4        DecisionTree  0.817012  0.806445
2           LinearSVC  0.716941  0.712363
1  LogisticRegression  0.687634  0.684622
0       MultinomialNB  0.649750  0.650457
5    GradientBoosting  0.647248  0.640926


In [22]:
best_overall = df_results.loc[df_results["macro_f1"].idxmax()]

print("🏆 Best model overall")
print(best_overall)


🏆 Best model overall
model       RandomForest
accuracy        0.875268
macro_f1        0.873013
Name: 3, dtype: object
